In [1]:
#Carga de Librerias
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import os

In [2]:
pur_prices_2017 = pd.read_csv("2017PurchasePricesDec.csv")

In [12]:
pur_prices_2017.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12256 entries, 0 to 12260
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Brand           12256 non-null  int64  
 1   Description     12256 non-null  object 
 2   Price           12256 non-null  float64
 3   Size            12256 non-null  object 
 4   Volume          12256 non-null  object 
 5   Classification  12256 non-null  int64  
 6   PurchasePrice   12256 non-null  float64
 7   VendorNumber    12256 non-null  int64  
 8   VendorName      12256 non-null  object 
 9   Size_ml         12256 non-null  float64
dtypes: float64(3), int64(3), object(4)
memory usage: 1.0+ MB


In [13]:
#Estadisticas de cada columna
for i in pur_prices_2017.columns:
    print(i)
    #Esto imprime las estadísticas descriptivas (como count, mean, std, min, 25%, 50%, 75%, max) para esa columna.
    print(pur_prices_2017[i].describe())
    print('--------------------------------------------------------------')

Brand
count    12256.000000
mean     17986.885362
std      12512.055720
min         58.000000
25%       5991.500000
50%      18793.500000
75%      25117.250000
max      90631.000000
Name: Brand, dtype: float64
--------------------------------------------------------------
Description
count                    12256
unique                   11110
top       Jagermeister Liqueur
freq                         7
Name: Description, dtype: object
--------------------------------------------------------------
Price
count    12256.000000
mean        38.651888
std        206.192338
min          0.000000
25%         10.990000
50%         15.990000
75%         29.990000
max      13999.900000
Name: Price, dtype: float64
--------------------------------------------------------------
Size
count     12256
unique       55
top       750mL
freq       9823
Name: Size, dtype: object
--------------------------------------------------------------
Volume
count     12256
unique       32
top         750
freq     

In [17]:
print(pur_prices_2017['Volume'].unique())
print('--------------------------------')
print(pur_prices_2017['Size'].unique())

[  750  1000  1750    50   375   100   200   300   250  1500  3000  5000
  4000   187   150   500   720   650   330 18000   180  6000    20 20000
   162   400  1100   600 19500   560  3750  9000]
--------------------------------
['750mL' '1000mL' '1750mL' '50mL' '375mL' '100mL 4 Pk' '50mL 5 Pk' '100mL'
 '200mL' '50mL 4 Pk' '50mL 3 Pk' '300mL' '200mL 4 Pk' '750mL 2 Pk'
 '250mL 4 Pk' '1500mL' '3000mL' '5000mL' '4000mL' '187mL 4 Pk' '150mL'
 '187mL' '500mL' '720mL' '650mL' '200mL 3 Pk' '187mL 3 Pk' '330mL' '250mL'
 '750mL + 3/' '18000mL' '180mL' '750mL + 4/' '6000mL' '1000mL 2 Pk'
 '750mL 12 P' '750mL 6 Pk' '20mL 5 Pk' '375mL 2 Pk' '20000mL' '50mL 12 Pk'
 '750mL 3 Pk' '375mL 3 Pk' '750mL + 1/' '200mL 5 Pk' '162.5mL' '400mL'
 '1100ml' '600mL' '19500mL' '560mL' '3750mL' '750mL  3' '1500ml' '9000mL']


In [9]:
import re

def convertir_a_ml(size):
    # Expresión regular para extraer el número principal, unidad y posibles cantidades de paquetes
    match = re.match(r"(\d+(\.\d+)?)\s*(mL|L|liter|oz)?(?:\s*(\d+)?\s*Pk)?", str(size), re.IGNORECASE)
    
    if match:
        valor = float(match.group(1))  # Valor numérico principal
        unidad = match.group(3).lower() if match.group(3) else ''  # Unidad de medida
        cantidad_pk = int(match.group(4)) if match.group(4) else 1  # Cantidad en el paquete, por defecto 1
        
        # Convertir a mililitros según la unidad
        if unidad in ['l', 'liter']:
            return valor * 1000 * cantidad_pk  # Convertir litros a mililitros y multiplicar por cantidad
        elif unidad == 'oz':
            return valor * 29.5735 * cantidad_pk  # Convertir onzas a mililitros y multiplicar por cantidad
        elif unidad == 'ml':
            return valor * cantidad_pk  # Ya está en mililitros, multiplicar por cantidad
    return None

# Aplicar la función a la columna 'Size' para crear la columna 'Size_ml' en mL
pur_prices_2017['Size_ml'] = pur_prices_2017['Size'].apply(convertir_a_ml)

# Verificar el resultado
print(pur_prices_2017[['Size', 'Size_ml']].head(10))

     Size  Size_ml
0   750mL    750.0
1   750mL    750.0
2   750mL    750.0
3   750mL    750.0
4   750mL    750.0
5   750mL    750.0
6   750mL    750.0
7  1000mL   1000.0
8  1000mL   1000.0
9   750mL    750.0


In [10]:
# Eliminar filas donde 'Size_ml' tiene valores NaN
pur_prices_2017 = pur_prices_2017.dropna(subset=['Size_ml'])

# Verificar el resultado
print(pur_prices_2017['Size_ml'].unique())

[  750.   1000.   1750.     50.    375.    400.    250.    100.    200.
   150.    300.    800.   1500.   3000.   5000.   4000.    748.    187.
   500.    720.    650.    600.    561.    330.  18000.    180.   6000.
  2000.   4500.  20000.   2250.   1125.    162.5  1100.  19500.    560.
  3750.   9000. ]


In [11]:

# Reemplaza 'Unknown' y None/NaN por NaN
pur_prices_2017['Volume'].replace(['Unknown', None], np.nan, inplace=True)
print(pur_prices_2017['Volume'].unique())


['750' '1000' '1750' '50' '375' '100' '200' '300' '250' '1500' '3000'
 '5000' '4000' '187' '150' '500' '720' '650' '330' '18000' '180' '6000'
 '20' '20000' '162.5' '400' '1100' '600' '19500' '560' '3750' '9000']


C:\Users\Admin\AppData\Local\Temp\ipykernel_6980\1884122741.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  pur_prices_2017['Volume'].replace(['Unknown', None], np.nan, inplace=True)


In [14]:
# Convierte a numérico, ignorando errores (convertirá los no numéricos a NaN)
pur_prices_2017['Volume'] = pd.to_numeric(pur_prices_2017['Volume'], errors='coerce')

print(pur_prices_2017['Volume'].describe())

count    12256.000000
mean       842.921467
std        679.529628
min         20.000000
25%        750.000000
50%        750.000000
75%        750.000000
max      20000.000000
Name: Volume, dtype: float64


In [16]:
# Elimina los decimales convirtiendo a entero
pur_prices_2017['Volume'] = pur_prices_2017['Volume'].astype(int)

print(pur_prices_2017['Volume'].info())

count    12256.000000
mean       842.921426
std        679.529669
min         20.000000
25%        750.000000
50%        750.000000
75%        750.000000
max      20000.000000
Name: Volume, dtype: float64


0         750
1         750
2         750
3         750
4         750
         ... 
12256     750
12257     750
12258     750
12259    1500
12260     750
Name: Volume, Length: 12256, dtype: int64


In [20]:
#Borrar columna Size porque ya se creo Size_ml

del pur_prices_2017['Size']

In [21]:
print(pur_prices_2017)

       Brand                   Description  Price  Volume  Classification  \
0         58   Gekkeikan Black & Gold Sake  12.99     750               1   
1         62      Herradura Silver Tequila  36.99     750               1   
2         63    Herradura Reposado Tequila  38.99     750               1   
3         72          No. 3 London Dry Gin  34.99     750               1   
4         75     Three Olives Tomato Vodka  14.99     750               1   
...      ...                           ...    ...     ...             ...   
12256  44917     Ferreira 10-Yr Tawny Port  24.99     750               2   
12257  44944     Sanford Santa Rita Pnt Nr  22.99     750               2   
12258  45016  Neal One Lane Bridg Cab Svgn  93.99     750               2   
12259  46011       Folonari Pnt Nr Venezie  12.99    1500               2   
12260  46964     Calera Viognier Mt Harlan  35.99     750               2   

       PurchasePrice  VendorNumber                   VendorName  Size_ml  


In [22]:
pur_prices_2017.to_csv("2017_purchases_prices_dec.csv")